# DES Crypt Demo

### Required
* pip install pycrypto
* pip install crcmod

## Example 1: Key Combinations
This example shows the number of key combinations based on the number of bits in a readable formation

In [ ]:
key_size_in_bits=112
"{:,}".format(2**key_size_in_bits)

## Example 2: DES
This example will show the normal usage of DES in a Python script.
Reference: http://www.umich.edu/~x509/ssleay/des-weak.html

In [ ]:
from Crypto.Cipher import DES
from Crypto import Random
import binascii
#Set the Initialization Vector (IV)
iv = Random.new().read(DES.block_size)
#Set the plaintext to be encrypted. It must be a multiple of 16
plaintext = b'ICS355 DES Test!' #Multiple of 16
#Set the key. It must be 8 bytes
skey="01fe01fe01fe01fe" #8 bytes
#The library requires a string byte array
key= str(bytearray.fromhex(skey))
#Create an instance of the cipher
cipher = DES.new(key, DES.MODE_OFB, iv)
#Encrypt the message. You all need to share the IV. You can append this to the message
msg = iv + cipher.encrypt(plaintext)
#msg = cipher.encrypt(plaintext)
#Print the encrypted message. This is what would be sent.
print "Encrypted message:\n", binascii.hexlify(msg)
#Get the IV
iv =msg[:DES.block_size]
#Get the encrypted message
encryptedmsg=msg[DES.block_size:]
#Create a new instance of the encryption library on the remote box
cipher = DES.new(key, DES.MODE_OFB, iv)
#Decrypt the message
mydecrypt= cipher.decrypt(encryptedmsg)
#Decypted message
print "Decrypted message:\n",mydecrypt

## Example 3:  DES Weak Keys
This shows an example of how not to use DES. A key of all 0s or all 1s results in a non-unique derived key. Unfortunately we are unable to see the internal keys since the Python library hides it from us. Note the second half of the output was decrypted with a different key.

In [ ]:
from Crypto.Cipher import DES
from Crypto import Random
import binascii
iv = Random.new().read(DES.block_size)
skey="ffffffffffffffff"
key= str(bytearray.fromhex(skey))
cipher = DES.new(key, DES.MODE_OFB, iv)
#msg = iv + cipher.encrypt(plaintext)
msg = cipher.encrypt(plaintext)
print "Encrypted message:\n", binascii.hexlify(msg)
skey2="0000000000000000"
#skey2="0101010101010101" #You can try this one too!
key2= str(bytearray.fromhex(skey2))
cipher2 = DES.new(key2, DES.MODE_OFB, iv)
mydecrypt= cipher2.decrypt(msg)
print "Decrypted message:\n",mydecrypt

## Example 4: DES Semi-Weak Keys
There are a handful of keys that can be used interchangeably which is something you never want to happen. 

For a symmetric algorithm you never want to see this situation: EK1(EK2(x))=x

Reference: http://www.umich.edu/~x509/ssleay/des-weak.html and http://crypto.stackexchange.com/questions/12214/can-you-explain-weak-keys-for-des 

Note: Must use ECB mode for demo

In [ ]:
from Crypto.Cipher import DES
from Crypto import Random
import binascii
#IV not needed for ECB mode
iv = Random.new().read(DES.block_size)
plaintext = b'ICS355 DES Test!' #Multiple of 16
desmode=DES.MODE_ECB #[DES.MODE_CBC, DES.MODE_CFB, DES.MODE_ECB, DES.MODE_OFB, DES.MODE_OPENPGP]: #DES.MODE_CTR requires a counter

SemiWeakKeys=[["01FE 01FE 01FE 01FE", "FE01 FE01 FE01 FE01"],
              ["1FE0 1FE0 0EF1 0EF1", "E01F E01F F10E F10E"],
              ["01E0 01E0 01F1 01F1", "E001 E001 F101 F101"],
              ["1FFE 1FFE 0EFE 0EFE", "FE1F FE1F FE0E FE0E"],
              ["011F 011F 010E 010E", "1F01 1F01 0E01 0E01"],
              ["E0FE E0FE F1FE F1FE", "FEE0 FEE0 FEF1 FEF1"]]
for key1,key2 in SemiWeakKeys:
    skey=key1
    key= str(bytearray.fromhex(skey))
    cipher = DES.new(key, desmode)
    #msg = iv + cipher.encrypt(plaintext)
    ctext = cipher.encrypt(plaintext)
    print "Key1 Ciphertext:\n",binascii.hexlify(ctext)
    skey=key2
    key= str(bytearray.fromhex(skey))
    cipher = DES.new(key, desmode)
    msg= cipher.encrypt(ctext)
    print "Key2: Plaintext:\n",msg
    print "\n"